In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.animation import FuncAnimation
from IPython import display


plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
def CalcABC(MOR, MOI, MSR, MSI, MLR, MLI):
    
    # Ideal open, short and load
    
    SOR = 1
    SOI = 0
    SSR = -1
    SSI = 0
    SLR = 0
    SLI = 0
       
    # Calculate temporary coefficients
    
    K1R = MLR - MSR
    K1I = MLI - MSI
    K2R = MSR - MOR
    K2I = MSI - MOI
    K3R = MOR - MLR
    K3I = MOI - MLI

    K4R = K1R*(SLR*SSR-SLI*SSI) - K1I*(SLR*SSI+SLI*SSR)
    K4I = K1R*(SLR*SSI+SLI*SSR) + K1I*(SLR*SSR-SLI*SSI)

    K5R = K2R*(SOR*SSR-SOI*SSI) - K2I*(SOR*SSI+SOI*SSR)
    K5I = K2R*(SOR*SSI+SOI*SSR) + K2I*(SOR*SSR-SOI*SSI)

    K6R = K3R*(SLR*SOR-SLI*SOI) - K3I*(SLR*SOI+SLI*SOR)
    K6I = K3R*(SLR*SOI+SLI*SOR) + K3I*(SLR*SOR-SLI*SOI)

    K7R = SOR*K1R - SOI*K1I
    K7I = SOR*K1I + SOI*K1R

    K8R = SLR*K2R - SLI*K2I
    K8I = SLR*K2I + SLI*K2R

    K9R = SSR*K3R - SSI*K3I
    K9I = SSR*K3I + SSI*K3R

    DR = K4R + K5R + K6R
    DI = K4I + K5I + K6I

    AnumR = MOR*K7R - MOI*K7I + MLR*K8R - MLI*K8I + MSR*K9R - MSI*K9I
    AnumI = MOR*K7I + MOI*K7R + MLR*K8I + MLI*K8R + MSR*K9I + MSI*K9R

    BnumR = MOR*K4R - MOI*K4I + MLR*K5R - MLI*K5I + MSR*K6R - MSI*K6I
    BnumI = MOR*K4I + MOI*K4R + MLR*K5I + MLI*K5R + MSR*K6I + MSI*K6R

    CnumR = K7R + K8R + K9R
    CnumI = K7I + K8I + K9I

    # Calculate A, B and C
    
    AR = (AnumR*DR + AnumI*DI)/(DR*DR + DI*DI)
    AI = (AnumI*DR - AnumR*DI)/(DR*DR + DI*DI)

    BR = (BnumR*DR + BnumI*DI)/(DR*DR + DI*DI)
    BI = (BnumI*DR - BnumR*DI)/(DR*DR + DI*DI)

    CR = (CnumR*DR + CnumI*DI)/(DR*DR + DI*DI)
    CI = (CnumI*DR - CnumR*DI)/(DR*DR + DI*DI)

    return AR, AI, BR, BI, CR, CI


In [3]:
def CalcMeasured(SR, SI, AR, AI, BR, BI, CR, CI):

    # Eq. 1
    
    MnumR = AR*SR - AI*SI + BR
    MnumI = AR*SI + AI*SR + BI
    MdenR = 1 + CR*SR - CI*SI
    MdenI = CR*SI + CI*SR

    MR = (MnumR*MdenR + MnumI*MdenI)/(MdenR*MdenR + MdenI*MdenI)
    MI = (MnumI*MdenR - MnumR*MdenI)/(MdenR*MdenR + MdenI*MdenI)

    return MR, MI

In [4]:
def CalcActual(MR, MI, AR, AI, BR, BI, CR, CI):

    # Eq. 2
    
    SnumR = MR - BR
    SnumI = MI - BI
    SdenR = AR + CI*MI - CR*MR
    SdenI = AI - CR*MI - CI*MR

    SR = (SnumR*SdenR + SnumI*SdenI)/(SdenR*SdenR + SdenI*SdenI)
    SI = (SnumI*SdenR - SnumR*SdenI)/(SdenR*SdenR + SdenI*SdenI)

    return SR, SI

In [5]:
def rotate(x, y, angle):
  angler = angle*np.pi/180
  newx = x*np.cos(angler) - y*np.sin(angler)
  newy = x*np.sin(angler) + y*np.cos(angler)
  return (newx, newy)

In [1]:
def plot_grid(x,y, **kwargs):    
    ax = plt.gca()
    segs1 = np.stack((x,y), axis=2)
    segs2 = segs1.transpose(1,0,2)
    ax.add_collection(LineCollection(segs1, **kwargs))
    ax.add_collection(LineCollection(segs2, **kwargs))

In [2]:
s = 1
p = 15
x = np.linspace(-s,s,p)
y = np.linspace(-s,s,p)
(x,y) = np.meshgrid(x,y)

NameError: name 'np' is not defined

In [8]:
def draw_graph_once(mor, moi, msr, msi, mlr, mli):

    ar,ai,br,bi,cr,ci = CalcABC(mor,moi,msr,msi,mlr,mli)
    u,v = CalcMeasured(x,y,ar,ai,br,bi,cr,ci)
    
    plot_grid(u,v, color="lightblue")
    plot_grid(x,y, color="pink")


    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.title("Reflection coefficient Γ")
    
    
    plt.quiver(u,v,x-u,y-v, angles='xy', scale_units='xy', scale=1., color='orange')
    
    circle1 = plt.Circle((0, 0), 1, color='grey', fill = False)
    plt.gca().add_patch(circle1)
    
    plt.axhline(y=0, color='grey', linestyle='-')
    plt.axvline(x=0, color='grey', linestyle='-')
    
    # plt.gca().set_aspect('equal')
    
    
    # markers = ['o', 's', '^', 'v', 'D', '*', '+', 'x']
   
    ms = 10
    plt.plot(mor, moi, marker='o', linestyle='', ms=ms)
    plt.plot(msr, msi, marker='o', linestyle='', ms=ms)
    plt.plot(mlr, mli, marker='o', linestyle='', ms=ms)
    
    plt.text(-1.4, 1.4, "Open: {0:.3f}{1}j{2:.3f}".format(mor, '+-'[int(moi < 0)], abs(moi)))
    plt.text(-1.4, 1.3, "Short: {0:.3f}{1}j{2:.3f}".format(msr, '+-'[int(msi < 0)], abs(msi)))
    plt.text(-1.4, 1.2, "Load: {0:.3f}{1}j{2:.3f}".format(mlr, '+-'[int(mli < 0)], abs(mli)))

    plt.text(0.7, 1.4, "a: {0:.3f}{1}j{2:.3f}".format(ar, '+-'[int(ai < 0)], abs(ai)))
    plt.text(0.7, 1.3, "b: {0:.3f}{1}j{2:.3f}".format(br, '+-'[int(bi < 0)], abs(bi)))
    plt.text(0.7, 1.2, "c: {0:.3f}{1}j{2:.3f}".format(cr, '+-'[int(ci < 0)], abs(ci)))

    # print("open={}+{}j  short={}+{}j load={}+{}j".format(mor, moi, msr, msi, mlr, mli))
    # print("a={}+{}j  b={}+{}j  c={}+{}j ".format(ar,ai,br,bi,cr,ci))
    



In [9]:
fig = plt.figure()

def animate(frame):
    fig.clear()

    # plt.axis("scaled")
    plt.xlim(-1.5, 1.5)
    plt.ylim(-1.5, 1.5)

    mor, moi = 1, 0
    msr, msi = -1, 0
    mlr, mli = 0, 0

    angle = 0
    
    if frame<=5:
        pass
    elif frame<=15:
        angle = -frame+5
    elif frame<=20:
        angle = -10
    elif frame<=30:
        angle = -10+frame-20
    elif frame<=35:
        pass        
    elif frame<=45:
        mor *= 1 + .01*(frame-35)
    elif frame<=50:
        mor *= 1.1  
    elif frame<=60:
        mor *= 1 + .01*(60-frame)
    elif frame<=65:
        pass   
    elif frame<=75:
        mor *= 1 - .01*(frame-65)
    elif frame<=80:
        mor *= 0.9  
    elif frame<=90:
        mor *= 1 - .01*(90-frame)
    elif frame<=95:
        pass   
    elif frame<=105:
        mor *= 1 + .01*(frame-95)
        msr *= 1 + .01*(frame-95)
    elif frame<=110:
        mor *= 1.1  
        msr *= 1.1  
    elif frame<=120:
        mor *= 1 + .01*(120-frame)
        msr *= 1 + .01*(120-frame)
    elif frame<=125:
        pass   
    elif frame<=135:
        angle = -frame+125
        mor += .03*(frame-125)
        moi += .03*(frame-125)
        mlr += .01*(frame-125)
    elif frame<=145:
        angle = -10
        mor += .3
        moi += .3
        mlr += .1     
    elif frame<=155:
        angle = -10+frame-145        
        mor += .03*(155-frame)
        moi += .03*(155-frame)
        mlr += .01*(155-frame)
        
    mor, moi = rotate(mor, moi, angle)
    msr, msi = rotate(msr, msi, angle)
    mlr, mli = rotate(mlr, mli, angle)
    
    draw_graph_once(mor, moi, msr, msi, mlr, mli)

anim = FuncAnimation(fig, animate, frames=170, interval=100)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

